<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
XAI Frameworks im Vergleich - Titanic
</b></font> </br></p>

---

**Explainable AI (XAI) Frameworks:**

- 🔍 **LIME** - Local Interpretable Model-agnostic Explanations
- 🎯 **SHAP** - SHapley Additive exPlanations  
- 👶 **ELI5** - Explain Like I'm 5
- 🏢 **InterpretML** - Microsoft's umfassendes Framework

Dieses Notebook vergleicht alle wichtigen XAI-Frameworks am Titanic-Datensatz.

---

**📚 Wichtige Begriffe für dieses Notebook:**

| Begriff | Bedeutung |
|---------|----------|
| **Black-Box-Modell** | ML-Modell, dessen Entscheidungslogik nicht direkt einsehbar ist (z.B. Random Forest, neuronale Netze) |
| **Modell-agnostisch** | Die Methode funktioniert bei jedem Modelltyp – egal ob Random Forest, XGBoost oder neuronales Netz |
| **Lokal vs. Global** | Lokal = erklärt eine einzelne Vorhersage; Global = erklärt das gesamte Modellverhalten |
| **Feature Importance** | Wie wichtig ist ein Merkmal für die Vorhersage? |
| **Perturbation** | Gezieltes Verändern von Eingabewerten, um deren Einfluss zu messen |


# 0  | Install & Import
---

In [ ]:
# Install - Verwendet normales pip (funktioniert in allen Umgebungen)
!uv pip install -q shap lime eli5 interpret plotly
!uv pip install -q git+https://github.com/parrt/dtreeviz.git

print("✅ Alle Pakete installiert")

In [ ]:
# Import
from pandas import read_excel, DataFrame
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# XAI Frameworks
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer
import eli5
from eli5.sklearn import PermutationImportance
from interpret import show
from interpret.blackbox import ShapKernel

import plotly.express as px
import matplotlib.pyplot as plt
import dtreeviz

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# Matplotlib-spezifische Warnungen unterdrücken
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)

# 1  | Understand
---

<p><font color='black' size="5">
📒 Anwendungsfall
</font></p>

Der legendäre Titanic ML-Wettbewerb: Vorhersage, welche Passagiere überlebt haben.

**Ziel**: Vergleich verschiedener XAI-Frameworks zur Erklärung der Modellvorhersagen.

**Beispielpersonen:**
- 👰 **Rose DeWitt Bukater** (Kate Winslet) - 1. Klasse, weiblich, 22 Jahre
- 🎨 **Jack Dawson** (Leonardo DiCaprio) - 3. Klasse, männlich, 23 Jahre


In [ ]:
df = read_excel(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/titanic.xlsx",
    usecols=["pclass", "survived", "sex", "age", "sibsp", "parch"],
)

data = df.copy()
target = data.pop("survived")

data.head()

#  2 | Prepare
---


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Train-Test-Split durchführen</br>
✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>


<font color='black' size="5">
Datentyp ermitteln
</font>


In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<font color='black' size="5">
Train-Test-Set
</font>


In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.20, stratify=target, random_state=42)

<font color='black' size="5">
Missing Values behandeln
</font>

In [ ]:
# Missing Values - numerische Features (z.B. age)
imputer_num = SimpleImputer(strategy="mean")
imputer_num.fit(data_train[num_col])
data_train[num_col] = imputer_num.transform(data_train[num_col])
data_test[num_col] = imputer_num.transform(data_test[num_col])

# Missing Values - kategoriale Features (z.B. sex)
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit(data_train[cat_col])
data_train[cat_col] = imputer_cat.transform(data_train[cat_col])
data_test[cat_col] = imputer_cat.transform(data_test[cat_col])

<font color='black' size="5">
Kodierung
</font>

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(data_train[cat_col])
data_train[cat_col] = encoder.transform(data_train[cat_col])
data_test[cat_col] = encoder.transform(data_test[cat_col])

# 3 | Modeling
---

<p><font color='black' size="5">
🏃 Modellauswahl & Training
</font></p>


In [ ]:
# RandomForestClassifier mit optimierten Hyperparametern (ohne Pipeline!)
# Reduziert Overfitting durch:
# - Mehr Bäume (n_estimators=200)
# - Weniger tiefe Bäume (max_depth=5)
# - Größere Mindestanzahl pro Blatt (min_samples_leaf=5)
# - Größere Mindestanzahl für Split (min_samples_split=10)

model = RandomForestClassifier(
    n_estimators=200,        # Mehr Bäume für stabilere Vorhersagen
    max_depth=5,             # Weniger tief = weniger Overfitting
    min_samples_split=10,    # Mind. 10 Samples für Split
    min_samples_leaf=5,      # Mind. 5 Samples pro Blatt
    max_features='sqrt',     # Nur Wurzel der Features pro Split
    random_state=42
)

model.fit(data_train, target_train)

# 4 | Evaluate
---

<p><font color='black' size="5">
🎯 Accuracy
</font></p>


In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

acc_train = accuracy_score(target_train, target_train_pred) * 100
acc_test = accuracy_score(target_test, target_test_pred) * 100

print(f"Train Accuracy: {acc_train:5.2f}%")
print(f"Test Accuracy:  {acc_test:5.2f}%")

<p><font color='black' size="5">
📝 Testpersonen: Rose & Jack
</font></p>

In [ ]:
# Rose DeWitt Bukater (Kate Winslet) - Original-Werte (keine Skalierung!)
rose = DataFrame(
    {"age": [22], "sex": [0], "sibsp": [0], "parch": [1], "pclass": [1]},  # sex=1 (female)
    index=["Rose"],
)

# Jack Dawson (Leonardo DiCaprio) - Original-Werte (keine Skalierung!)
jack = DataFrame(
    {"age": [23], "sex": [1], "sibsp": [0], "parch": [0], "pclass": [3]},  # sex=0 (male)
    index=["Jack"],
)

rose_pred = model.predict_proba(rose)[0][1] * 100
jack_pred = model.predict_proba(jack)[0][1] * 100

print(f"👰 Rose Überlebenschance: {rose_pred:.2f}%")
print(f"   (22 Jahre, weiblich, 1. Klasse)")
print(f"🎨 Jack Überlebenschance: {jack_pred:.2f}%")
print(f"   (23 Jahre, männlich, 3. Klasse)")

<p><font color='black' size="5">
🌳 Entscheidungsbaum Visualisierung
</font></p>

**Hinweis**: RandomForest besteht aus 200 Bäumen. Hier wird nur der ersten Baum als Beispiel visualisiert.


In [ ]:
# Extrahiere den ersten Baum aus dem RandomForest
single_tree = model.estimators_[1]

# Erstelle dtreeviz Visualisierung für diesen einzelnen Baum
viz_model = dtreeviz.model(
    single_tree,
    X_train=data_train,
    y_train=target_train,
    target_name="survived",
    class_names=["not survived", "survived"],
    feature_names=list(data_train.columns),
)

print(f"✅ Visualisierung für Baum 1 von {model.n_estimators} erstellt")

In [ ]:
# Visualisierung für diesen einzelnen Baum
viz_model.view(scale=1.0, fontname="Monospace")

In [ ]:
# Local Explanation für Rose - total
one = rose.iloc[0].values
viz_model.view(x=one, scale=1.0, fontname="Monospace")

In [ ]:
# local Explanation - single
viz_model.view(x=one, scale=1.2, show_just_path=True, fontname="Monospace")

# 5 | Deploy
---

# A | XAI mit LIME 🔍
---

**LIME** = Local Interpretable Model-agnostic Explanations

**Was macht LIME?**

LIME erklärt **einzelne Vorhersagen**, indem es ein einfaches, verständliches Modell lokal um den zu erklärenden Datenpunkt herum trainiert.

**Wie funktioniert LIME?**

1. **Perturbation**: LIME erzeugt viele leicht veränderte Versionen des Datenpunkts (z.B. Alter ±5 Jahre, andere Klasse)
2. **Vorhersagen sammeln**: Das Black-Box-Modell bewertet alle perturbierten Samples
3. **Gewichtung**: Samples, die dem Original ähnlicher sind, bekommen mehr Gewicht
4. **Lokales Modell**: Ein einfaches lineares Modell wird auf diese gewichteten Samples trainiert
5. **Interpretation**: Die Koeffizienten des linearen Modells zeigen den Einfluss jedes Features

**Warum "lokal"**?

LIME erklärt nicht das gesamte Modell, sondern nur die **unmittelbare Umgebung** eines Datenpunkts. Die Erklärung für Rose kann völlig anders aussehen als die für Jack.

**Vorteile**
- ✅ Funktioniert mit **jedem** Modell (modell-agnostisch)
- ✅ Sehr **intuitiv** zu verstehen
- ✅ **Schnell** für einzelne Vorhersagen


In [ ]:
# LIME Explainer mit vorverarbeiteten Daten erstellen
lime_explainer = LimeTabularExplainer(
    data_train.values,
    feature_names=data_train.columns.tolist(),
    class_names=['Not Survived', 'Survived'],
    categorical_features=[data_train.columns.get_loc('sex'), data_train.columns.get_loc('pclass')],
    mode='classification'
)

print("✅ LIME Explainer erstellt")
print("   Features:", data_train.columns.tolist())

<p><font color='black' size="5">
👰 Rose mit LIME erklären
</font></p>


In [ ]:
# Rose mit LIME erklären
rose_lime_exp = lime_explainer.explain_instance(
    rose.iloc[0].values,
    model.predict_proba,
    num_features=5
)
rose_lime_exp.show_in_notebook(show_table=True)

<p><font color='black' size="5">
🎨 Jack mit LIME erklären
</font></p>

In [ ]:
# Jack mit LIME erklären
jack_lime_exp = lime_explainer.explain_instance(
    jack.iloc[0].values,
    model.predict_proba,
    num_features=5
)

jack_lime_exp.show_in_notebook(show_table=True)

<p><font color='blue' size="4">
💡 LIME Interpretation
</font></p>

**Balkendiagramm:**
- **Orange Balken**: Erhöhen die Überlebenschance (Klasse "Survived")
- **Blaue Balken**: Verringern die Überlebenschance (Klasse "Not Survived")
- **Werte**: Wie stark beeinflusst das Feature die Vorhersage

**PrScore (Prediction Score) Tabelle:**
- Zeigt die Vorhersage-Wahrscheinlichkeit für **beide Klassen**
- **Not Survived**: Wahrscheinlichkeit für Tod (höher bei Jack)
- **Survived**: Wahrscheinlichkeit für Überleben (höher bei Rose)
- Beide Werte zusammen ergeben immer 100%

**Beispiel:**
- Rose: Not Survived ≈ 38%, **Survived ≈ 62%**
- Jack: **Not Survived ≈ 93%**, Survived ≈ 7%

⚠️ **Wichtig**: Der PrScore für "Not Survived" ist bei Jack höher - das bedeutet er hat eine höhere Todeswahrscheinlichkeit, nicht eine höhere Überlebenschance!

**Erkenntnisse:**
- Rose: `sex=0` (female) und `pclass=1` erhöhen massiv die Überlebenschance
- Jack: `sex=1` (male) und `pclass=3` verringern massiv die Überlebenschance


# B | XAI mit SHAP 🎯
---

**SHAP** = SHapley Additive exPlanations

**Was macht SHAP?**

SHAP berechnet den **Beitrag jedes Features** zur Vorhersage, basierend auf einem mathematisch fundierten Konzept aus der Spieltheorie.

**Die Spieltheorie-Idee (vereinfacht**)

Stellen Sie sich ein Team von Spielern vor, das gemeinsam einen Preis gewinnt. Wie verteilt man den Gewinn **fair**? Die Shapley-Werte lösen genau dieses Problem.

**Übertragen auf ML:**
- **Spieler** = Features (Alter, Geschlecht, Klasse, ...)
- **Gewinn** = Vorhersage des Modells
- **Frage**: Wie viel trägt jedes Feature zur Vorhersage bei?

**Wie funktioniert SHAP**?

1. Betrachte **alle möglichen Kombinationen** von Features
2. Berechne für jede Kombination: Was ändert sich, wenn ich Feature X hinzufüge?
3. Der **Durchschnitt** über alle Kombinationen = SHAP-Wert des Features

**Interpretation der SHAP-Werte**

| SHAP-Wert | Bedeutung |
|-----------|----------|
| **Positiv (+)** | Feature erhöht die Vorhersage (hier: Überlebenschance) |
| **Negativ (-)** | Feature senkt die Vorhersage |
| **Nahe 0** | Feature hat wenig Einfluss auf diese Vorhersage |

**Vorteile gegenüber LIME**
- ✅ **Theoretisch fundiert** – mathematisch beweisbar fair
- ✅ Funktioniert **lokal und global**
- ✅ Die Summe aller SHAP-Werte ergibt die Vorhersage


<p><font color='black' size="5">
⚙️ SHAP Explainer erstellen
</font></p>

In [ ]:
# SHAP TreeExplainer (optimal für RandomForest - sehr schnell!)
shap_explainer = shap.TreeExplainer(model)

print("✅ SHAP TreeExplainer erstellt (optimiert für RandomForest)")

<p><font color='black' size="5">
👰 Rose mit SHAP erklären (Waterfall Plot)
</font></p>


In [ ]:
rose_shap = shap_explainer(rose)

# Waterfall Plot für Klasse "Survived" (Index 1)
shap.plots.waterfall(rose_shap[0, :, 1])

<p><font color='black' size="5">
🎨 Jack mit SHAP erklären (Waterfall Plot)
</font></p>

In [ ]:
jack_shap = shap_explainer(jack)

# Waterfall Plot für Klasse "Survived" (Index 1)
shap.plots.waterfall(jack_shap[0, :, 1])

<p><font color='blue' size="4">
💡 SHAP Interpretation
</font></p>

**Waterfall Plot:**
- Zeigt, wie sich die Vorhersage vom Durchschnittswert (E[f(x)]) zum finalen Wert aufbaut
- Jeder Pfeil = Beitrag eines Features
- Rot = erhöht die Überlebenschance, Blau = verringert sie

# C | XAI mit ELI5 👶
---

**ELI5** = Explain Like I'm 5 (Erkläre es mir wie einem 5-Jährigen)

**Was macht ELI5?**

ELI5 ist das **einfachste** XAI-Framework. Es fokussiert sich auf **Permutation Importance** – eine intuitive Methode zur Messung der Feature-Wichtigkeit.

**Was ist Permutation Importance?**

Die Grundidee ist simpel: **Wenn ein Feature wichtig ist, wird das Modell schlechter, wenn wir es "kaputt machen".**

**So funktioniert es:**
1. Miss die Modell-Accuracy auf den Testdaten
2. Mische die Werte eines Features zufällig durch ("permutieren")
3. Miss erneut die Accuracy
4. **Differenz** = Wichtigkeit des Features

**Beispiel:**
- Original-Accuracy: 85%
- Accuracy nach Durchmischen von "Geschlecht": 65%
- → Importance(Geschlecht) = 85% - 65% = **20%** (sehr wichtig!)

**Warum "Like I'm 5"?**

- Minimaler Code (oft nur 3 Zeilen)
- Keine komplexe Mathematik
- Ergebnis ist sofort verständlich: "Feature X ist am wichtigsten"

**Einschränkungen**
- ⚠️ Zeigt nur **wie wichtig**, nicht **warum** oder **in welche Richtung**
- ⚠️ Primär für **globale** Erklärungen (nicht für einzelne Vorhersagen)


<p><font color='black' size="5">
⚙️ Permutation Importance
</font></p>

In [ ]:
# Permutation Importance berechnen
perm = PermutationImportance(model, random_state=42).fit(data_test, target_test)

In [ ]:
# Feature Importance anzeigen
eli5.show_weights(perm, feature_names=data.columns.tolist())

<p><font color='black' size="5">
👰 Rose mit ELI5 erklären
</font></p>


In [ ]:
# Einzelne Vorhersage erklären
eli5.show_prediction(
    model,
    rose.iloc[0],
    feature_names=rose.columns.tolist(),
    show_feature_values=True,
)

<p><font color='black' size="5">
🎨 Jack mit ELI5 erklären
</font></p>


In [ ]:
# Einzelne Vorhersage erklären
eli5.show_prediction(
    model,
    jack.iloc[0],
    feature_names=jack.columns.tolist(),
    show_feature_values=True,
    targets=[1]  # 1 = Überleben
)

<p><font color='blue' size="4">
💡 ELI5 Interpretation
</font></p>



**Permutation Importance:**
- Misst, wie stark die Modell-Accuracy sinkt, wenn ein Feature zufällig permutiert wird
- ± Werte = Unsicherheit der Messung

**Vorhersage-Erklärung:**
- Zeigt Feature-Werte und ihre Beiträge zur Vorhersage
- Sehr einfach zu verstehen!



# D | XAI mit InterpretML 🏢
---

**InterpretML** = Microsoft's umfassendes Open-Source-Framework für XAI

**Was macht InterpretML?**

InterpretML ist das **professionellste** der hier vorgestellten Frameworks. Es kombiniert verschiedene XAI-Methoden unter einer einheitlichen Oberfläche und bietet **interaktive Dashboards**.

**Kernfunktionen**

| Funktion | Beschreibung |
|----------|-------------|
| **ShapKernel** | SHAP-Erklärungen für beliebige Black-Box-Modelle |
| **Interaktive Dashboards** | Web-basierte Visualisierungen zum Erkunden |
| **Unified API** | Gleiche Schnittstelle für verschiedene Erklärungsmethoden |
| **EBM** | Eigenes interpretierbares Modell (Explainable Boosting Machine) |

**Wann InterpretML verwenden**?

- ✅ Wenn Sie **interaktive Exploration** brauchen
- ✅ Für **professionelle Präsentationen** und Berichte
- ✅ Wenn Sie **verschiedene XAI-Methoden** vergleichen wollen

**Einschränkungen**
- ⚠️ Mehr Setup-Aufwand als LIME oder ELI5
- ⚠️ Kann für einfache Aufgaben "überdimensioniert" sein


<p><font color='black' size="5">
⚙️ InterpretML mit SHAP
</font></p>


In [ ]:
# SHAP-basierter Explainer in InterpretML
interpret_explainer = ShapKernel(
    model.predict_proba,
    data_train.sample(n=100, random_state=42)
)

print("✅ InterpretML Explainer erstellt")

<p><font color='black' size="5">
👰🎨 Rose & Jack mit InterpretML
</font></p>

**Was macht `explain_local`?**
- Erklärt **lokale** Vorhersagen für einzelne Instanzen (hier: Rose & Jack)
- Verwendet SHAP-Werte im Hintergrund
- Zeigt Feature-Beiträge für jede Person einzeln
- Ermöglicht interaktiven Vergleich zwischen Instanzen


In [ ]:
# Beide Personen kombinieren
rose_jack = DataFrame([rose.iloc[0], jack.iloc[0]], index=["Rose", "Jack"])
rose_jack_target = [1, 0]  # Rose überlebt, Jack nicht

In [ ]:
# Lokale Erklärungen erstellen mit Namen
# explain_local liefert:
# - Feature-Beiträge für jede einzelne Instanz
# - Interaktive Visualisierung zum Vergleichen
# - Basiert auf SHAP-Werten
interpret_local = interpret_explainer.explain_local(
    rose_jack,
    rose_jack_target,
    name="Rose & Jack Erklärungen"  # Name für die Erklärung
)

# Die Namen werden aus dem DataFrame-Index übernommen
print("✅ Lokale Erklärungen für Rose & Jack erstellt")
print("   - Zeigt Feature-Beiträge für jede Person")
print("   - Interaktiv vergleichbar im Dashboard")
print("   - Namen: Rose (Index 0), Jack (Index 1)")

**⚠️ Wichtiger Hinweis zur Auswahl:**       
Im Dropdown "Select Component to Graph" bedeuten:
- **0** = Rose (1. Klasse, weiblich, 22 Jahre)
- **1** = Jack (3. Klasse, männlich, 23 Jahre)

In [ ]:
# ⚠️ WICHTIG: InterpretML benötigt JavaScript!
# Falls Fehler "JavaScript-Dateien konnten nicht geladen werden":
#
# 1. Chrome: chrome://settings/content/cookies
#    → "Drittanbieter-Cookies zulassen" aktivieren
# 2. Firefox: about:preferences#privacy
#    → Verbesserter Schutz: "Standard" statt "Streng"
# 3. Seite neu laden (Strg+R / Cmd+R)
#
# Alternative: Verwenden Sie SHAP (Cells oben) - ähnliche Ergebnisse!

# Interaktives Dashboard anzeigen
show(interpret_local)

# E | Globale Analysen 🌍
---

**Lokal vs. Global – Was ist der Unterschied?**

Bisher haben wir **einzelne Vorhersagen** erklärt (lokal). Jetzt schauen wir uns das **gesamte Modellverhalten** an (global).

| Scope | Frage | Beispiel |
|-------|-------|----------|
| **Lokal** | Warum wurde *diese* Vorhersage gemacht? | Warum überlebt Rose mit 92%? |
| **Global** | Wie verhält sich das Modell *insgesamt*? | Welches Feature ist generell am wichtigsten? |



**Globale Methoden in diesem Notebook**

- **SHAP Summary Plot**: Zeigt die Verteilung der SHAP-Werte über alle Datenpunkte
- **SHAP Dependence Plot**: Zeigt, wie ein Feature die Vorhersage über alle Datenpunkte beeinflusst
- **RandomForest Feature Importance**: Eingebaute Wichtigkeit basierend auf Entscheidungsbäumen


<p><font color='black' size="5">
🎯 SHAP Dependence Plot
</font></p>

**Konzept**: Zeigt, wie die Werte eines Features die Vorhersage beeinflussen.


In [ ]:
# SHAP Dependence Plot für das wichtigste Feature (sex)
# Zeigt: Wie beeinflusst sex die Überlebenschance?

# Berechne SHAP Werte für das gesamte Testset
test_sample = data_test
shap_values_test = shap_explainer(test_sample)

shap.dependence_plot(
    "sex",
    shap_values_test[:, :, 1].values,  # SHAP-Werte für "Survived"
    test_sample,
    interaction_index="pclass"  # Farbe zeigt Interaktion mit pclass
)

print("💡 Interpretation:")
print("   - X-Achse: Feature-Wert (0=male, 1=female)")
print("   - Y-Achse: SHAP-Wert (Einfluss auf Überlebenschance)")
print("   - Farbe: Interaktion mit pclass (Passagierklasse)")

<p><font color='black' size="5">
🌲 RandomForest Feature Importance
</font></p>

**Konzept**: Zeigt, wie oft und wie stark jedes Feature zur Entscheidungsfindung beiträgt.


In [ ]:
# Feature Importance aus RandomForest extrahieren
feature_importance = DataFrame({
    'Feature': data_train.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualisierung
fig = px.bar(
    feature_importance,
    x='Importance',
    y='Feature',
    orientation='h',
    title='RandomForest Feature Importance (Global)',
    labels={'Importance': 'Wichtigkeit', 'Feature': 'Feature'},
    color='Importance',
    color_continuous_scale='viridis'
)
fig.update_layout(height=400)
fig.show()

print("\n📊 Feature Importance Ranking:")
for idx, row in feature_importance.iterrows():
    print(f"  {row['Feature']:10s}: {row['Importance']:.4f}")

# F | Ceteris Paribus Analysen 🌍
---

**Ceteris Paribus** = "unter sonst gleichen Bedingungen" (lateinisch)

**Was ist eine Ceteris Paribus Analyse?**

Eine Ceteris Paribus Analyse beantwortet die Frage: **"Was passiert mit der Vorhersage, wenn ich NUR ein Feature ändere?"**

Alle anderen Features bleiben dabei konstant – daher der Name.

**Beispiel**

Für Jack (männlich, 23 Jahre, 3. Klasse) fragen wir:
- Was wäre, wenn Jack in der **1. Klasse** gewesen wäre? (alle anderen Werte bleiben gleich)
- Was wäre, wenn Jack **50 Jahre** alt gewesen wäre?

**Warum ist das nützlich?**

- 🎯 Zeigt den **isolierten Einfluss** einzelner Features
- 📊 Ermöglicht **"Was-wäre-wenn"**-Szenarien
- 💡 Hilft zu verstehen, **wie das Modell "denkt"**

**Unterschied zu anderen XAI-Methoden**

| Methode | Frage |
|---------|-------|
| **SHAP/LIME** | Welche Features haben diese Vorhersage beeinflusst? |
| **Ceteris Paribus** | Wie ändert sich die Vorhersage, wenn ich ein Feature variiere? |

**Erkenntnisse:**
1. **Alter**: Jüngere Personen hatten tendenziell höhere Überlebenschancen ("Women and children first")
2. **Passagierklasse**: 1. Klasse hatte deutlich höhere Überlebenschancen
3. **Geschlecht dominiert**: Selbst Jack in 1. Klasse hätte schlechtere Chancen als Rose in 3. Klasse!


<p><font color='black' size="5">
🎨 Individual Ceteris Paribus: Jack
</font></p>

**Was wäre wenn**: Jack in verschiedenen Passagierklassen gereist wäre?

In [ ]:
# Ceteris Paribus für Jack: Was passiert in verschiedenen Klassen?
jack_cp = jack.copy()

pclass_original = [1, 2, 3]
jack_predictions = []

for pclass in pclass_original:
    jack_cp['pclass'] = pclass
    pred = model.predict_proba(jack_cp)[0][1] * 100
    jack_predictions.append(pred)

# Visualisierung
fig = px.bar(
    x=['1. Klasse', '2. Klasse', '3. Klasse (Jack)'],
    y=jack_predictions,
    labels={'x': 'Passagierklasse', 'y': 'Überlebenschance (%)'},
    title='Ceteris Paribus: Jack - Einfluss der Passagierklasse',
    color=jack_predictions,
    color_continuous_scale='RdYlGn'
)
fig.show()

print("💡 Interpretation:")
print(f"   Jack in 1. Klasse: {jack_predictions[0]:.2f}%")
print(f"   Jack in 2. Klasse: {jack_predictions[1]:.2f}%")
print(f"   Jack in 3. Klasse: {jack_predictions[2]:.2f}% (aktuell)")
print("\n   ⚠️ Selbst in 1. Klasse würde Jack's männliches Geschlecht seine Chancen stark begrenzen!")

<p><font color='black' size="5">
👰 Individual Ceteris Paribus: Rose
</font></p>

**Was wäre wenn**:  Was passiert, wenn wir ihr Alter variieren??

In [ ]:
# Ceteris Paribus für Rose: Was passiert, wenn wir ihr Alter variieren?
# Alle anderen Features bleiben konstant

age_range = range(0, 80,1)  # Skalierte Werte (StandardScaler)
rose_cp = rose.copy()

predictions = []
for age_val in age_range:
    rose_cp['age'] = age_val
    pred = model.predict_proba(rose_cp)[0][1] * 100
    predictions.append(pred)

# Visualisierung
fig = px.line(
    x=age_range,
    y=predictions,
    labels={'x': 'Alter', 'y': 'Überlebenschance (%)'},
    title='Ceteris Paribus: Rose - Einfluss des Alters'
)
fig.add_vline(x=rose.iloc[0]['age'], line_dash="dash", line_color="red",
              annotation_text="Rose aktuell")
fig.show()

print(f"💡 Rose's aktuelles Alter (skaliert): {rose.iloc[0]['age']:.2f}")
print(f"   Aktuelle Überlebenschance: {rose_pred:.2f}%")
print(f"   Interpretation: Je jünger Rose, desto höher ihre Überlebenschance")

# G | Zusammenfassung 🔬
---

| Framework | 🎯 Stärken | ⚠️ Schwächen | 👨‍🎓 Einsteigerfreundlichkeit |
|-----------|-----------|-------------|---------------------------|
| **LIME** | - Sehr intuitives Konzept<br>- Gute visuelle Darstellung<br>- Schnell für lokale Erklärungen | - Nur lokale Erklärungen<br>- Kann instabil sein | ⭐⭐⭐⭐⭐ |
| **SHAP** | - Theoretisch fundiert<br>- Beste Visualisierungen<br>- Lokal & global | - Kann langsam sein<br>- Komplexeres Konzept | ⭐⭐⭐⭐ |
| **ELI5** | - Extrem einfach<br>- Minimaler Code<br>- Schnell | - Weniger Visualisierungen<br>- Weniger Features | ⭐⭐⭐⭐⭐ |
| **InterpretML** | - Interaktive Dashboards<br>- Umfassend<br>- Professionell | - Komplexer Setup<br>- Overhead für einfache Aufgaben | ⭐⭐⭐ |
| **RF Importance** | - Extrem schnell<br>- In sklearn integriert<br>- Sehr einfach | - Nur Feature Importance<br>- Keine Richtung des Einflusses | ⭐⭐⭐⭐⭐ |



**🔗 Weiterführende Ressourcen**

**LIME:**
- Dokumentation: https://github.com/marcotcr/lime
- Paper: "Why Should I Trust You?" (Ribeiro et al., 2016)

**SHAP:**
- Dokumentation: https://shap.readthedocs.io/
- Paper: "A Unified Approach to Interpreting Model Predictions" (Lundberg & Lee, 2017)

**ELI5:**
- Dokumentation: https://eli5.readthedocs.io/

**InterpretML:**
- Dokumentation: https://interpret.ml/
- Microsoft Research: https://www.microsoft.com/en-us/research/project/interpretml/
"""